In [ ]:
!pip install kagglehub

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mkechinov/ecommerce-behavior-data-from-multi-category-store")

print("Path to dataset files:", path)

In [ ]:
import os
#path = "C:/Users/User/.cache/kagglehub/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store/versions/8"
os.listdir(path)

In [ ]:

path2 = os.path.join(path, '2019-Oct.csv')

In [ ]:
import dask.dataframe as dd

# Dask 데이터프레임 생성
ddf = dd.read_csv(path2)

# 랜덤으로 1% 샘플링
sampled_data = ddf.sample(frac=0.01, random_state=42).compute()

print(sampled_data)

In [ ]:
print(sampled_data.head())
print(sampled_data.shape)
print(sampled_data.info())
print(sampled_data.columns)
visitor = sampled_data['user_id'].nunique()
print("visitors: {}".format(visitor))

In [ ]:

sampled_data['date'] = pd.to_datetime(sampled_data['event_time'].str.split(" ").str[0])

In [ ]:
sampled_data.head()

In [ ]:
print([(column,sampled_data[column].nunique()) for column in sampled_data.columns])

In [ ]:
visitor_by_date = sampled_data[['user_id', 'date']].drop_duplicates().groupby('date').count().sort_values(by='date', ascending=True)

In [ ]:
visitor_by_date

###날짜별 고객수 추이

In [ ]:
import matplotlib.pyplot as plt
x= pd.Series(visitor_by_date.index.values)
y= visitor_by_date["user_id"]
plt.rcParams["figure.figsize"]=(20,8)
plt.plot(x,y)
plt.xticks(rotation = 45)

In [ ]:
product_by_date = sampled_data[['product_id', 'date']].groupby('date').count().sort_values(by='date', ascending=True) # 날짜별 product 갯수
price_by_date = sampled_data[['price', 'date']].groupby('date').mean().sort_values(by='date', ascending=True) # 날짜별 price 평균

### 날짜별 총판매 추이


In [ ]:
total_profit=pd.DataFrame(product_by_date['product_id']*price_by_date['price'])
total_profit.columns=['total_profit']
plt.rcParams["figure.figsize"]=(20,8)
plt.plot(total_profit.index.values, total_profit['total_profit'])

In [ ]:
purchase_data = sampled_data.loc[sampled_data['event_type'] == 'purchase',:]

In [ ]:
purchase_data.reset_index(drop=True).head(5)

In [ ]:
product_price_map = purchase_data.groupby('product_id')['price'].mean().reset_index()


In [ ]:
product_price_map

In [ ]:
top_10_df = purchase_data.groupby('product_id').count().reset_index()[['product_id', 'date']]

In [ ]:
top_10_df.columns = ['product_id', 'count']
top_10_df

In [ ]:
top_10_df = pd.merge(top_10_df, product_price_map, on='product_id', how='outer')
top_10_df['total_profit'] = top_10_df['count'] * top_10_df['price']
top_10_df.sort_values(by='total_profit', ascending=False, inplace=True)
top_10_df.head(10)

In [ ]:
top_10_df.isnull().sum()
top_10_df.reset_index(drop=True, inplace=True)

In [ ]:
for product_id in top_10_df.loc[:10,"product_id"]:
    product_daily_price = purchase_data.loc[purchase_data['product_id'] == product_id,['event_date','price']].groupby(['event_date']).mean()
    product_daily_price = purchase_data[['event_date','price']].groupby(['event_date']).mean()

In [ ]:
purchase_data

In [ ]:
top_10_df.loc[:10,"product_id"]

In [ ]:
for product_id in top_10_df.loc[:10,"product_id"]:
    product_daily_price = purchase_data.loc[purchase_data['product_id'] == product_id, ['date', 'price']].groupby(['date']).mean()

        # x, y 값 생성
    x = pd.Series(product_daily_price.index.values)
    y = product_daily_price['price']

        # 각 product_id의 데이터를 그래프에 추가
    plt.plot(x, y, label=f"Product ID: {product_id}", marker='o')

# 그래프 설정
plt.title("Price Trends for Multiple Product IDs", fontsize=16)
plt.xlabel("Event Date", fontsize=14)
plt.ylabel("Price", fontsize=14)
plt.xticks(rotation=45)
plt.legend(title="Product ID", fontsize=12)
plt.grid(True)
# 그래프 출력
plt.show()